In [ ]:
# Stable release from PyPI:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048      # How many tokens each input can have
dtype = None               # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True        # Use 4bit quantization to reduce memory usage. Can be False.
model_name = "unsloth/Llama-3.2-1B-Instruct"
# Load both model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,          # Adapter rank (size), Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "up_proj", "down_proj"
    ],
    lora_alpha=16,                  # Scales adapter updates
    lora_dropout=0,                 # No dropout/ Supports any, but = 0 is optimized
    bias="none",                    # Skip bias updates / Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth", # True or "unsloth" for very long context
    random_state=3407,
    use_rslora = False,  # (Optional)
    loftq_config = None, # (Optional)
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.7 patched 16 layers with 16 QKV layers, 16 O layers and 0 MLP layers.


In [ ]:
# --- Configuration & Imports ---
# Choose whether to use a local CSV or an existing ShareGPT dataset
USE_CSV    = True                 # ← set False to load a ShareGPT dataset instead
CSV_PATH   = "/content/eastern_caverns_finetune_data.csv"      # ← CSV needs 'question' & 'answer' columns, Comment out if not needed
SHAREGPT_DS = "mlabonne/FineTome-100k"  # Any HF dataset in ShareGPT style, Comment out if not needed

# Core imports
import pandas as pd
from datasets import Dataset, load_dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

In [ ]:
# --- Load & Wrap CSV into ShareGPT format (if USE_CSV) ---
if USE_CSV:
    # Load your CSV into a DataFrame and then a Hugging Face Dataset
    df = pd.read_csv("/content/drive/MyDrive/eastern_caverns_finetune_data.csv")
    ds = Dataset.from_pandas(df)
    # Convert each Q&A row into a list of messages
    def to_sharegpt_format(ex):
        return {
            "conversations": [
                {"from": "system",  "value": "You are a Victorian-era assistant for a company named Eastern Caverns, a automation tech company"},
                {"from": "human",   "value": ex["question"]},
                {"from": "gpt",     "value": ex["answer"]},
            ]
        }
    ds = ds.map(to_sharegpt_format, remove_columns=df.columns.tolist())
else:
    # Load a ShareGPT-style dataset directly
    ds = load_dataset(SHAREGPT_DS, split="train")

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
# --- Standardize & Apply Chat Template ---
# Convert 'from'/'value' → 'role'/'content'
ds = standardize_sharegpt(ds)

# Pick your model's chat template (e.g., "llama-3.1" or "phi-4")
CHAT_TEMPLATE = "llama-3.1"
tokenizer = get_chat_template(tokenizer, chat_template=CHAT_TEMPLATE)
# Build the final text examples for fine-tuning
def format_prompts(examples):
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        )
        for convo in examples["conversations"]
    ]
    return {"text": texts}
# Apply to your dataset
nds = ds.map(format_prompts, batched=True)
# Now `nds["text"]` is ready for training!

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
# Inspect the result
print("=================BEFORE=====================")
print(nds[0]['conversations'])
# Inspect the result
print("=================AFTER=====================")
print(nds[0]['text'])

=================BEFORE=====================
[{'content': 'You are a Victorian-era assistant for a company named Eastern Caverns, a automation tech company', 'role': 'system'}, {'content': 'What is the primary function of the AetherSync-3000?', 'role': 'user'}, {'content': 'The AetherSync-3000 synchronizes heterogeneous automation protocols across multiple device ecosystems, ensuring real-time coordination.', 'role': 'assistant'}]
=================AFTER=====================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are a Victorian-era assistant for a company named Eastern Caverns, a automation tech company<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the primary function of the AetherSync-3000?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The AetherSync-3000 synchronizes heterogeneous automation protocols across multiple device ecosystems, ensuring real-time coordination.<|eo

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=nds,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps=100,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        output_dir="outputs",
        report_to="none",
    ),
)



Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part    = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 34 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,650,752/1,000,000,000 (0.87% trained)


Step,Training Loss
1,3.796300
2,4.475500
3,4.016300
4,7.373700
5,3.529700
6,3.943500
7,5.905400
8,3.000700
9,2.545200
10,5.166000


In [ ]:
#wrap your fine-tuned model for quick reply
model = FastLanguageModel.for_inference(model)
#prompting and loading to GPU
inputs = tokenizer.apply_chat_template(
    [{"role":"user","content":"With reference to eastern caverns company, Describe the load balancing mechanism in CavernFlow."}],
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

#Generate an answer:
outputs = model.generate(input_ids=inputs, max_new_tokens=256)
print(tokenizer.batch_decode(outputs)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

With reference to eastern caverns company, Describe the load balancing mechanism in CavernFlow.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

CavernFlow uses a weighted round-robin algorithm with health-check failover across EC-Worker nodes.<|eot_id|>


In [ ]:
# Save
#model.save_pretrained("/content/drive/MyDrive/my_llama3_model_eastern_caverns")
#tokenizer.save_pretrained("/content/drive/MyDrive/my_llama3_model_eastern_caverns")
# Reload in 4-bit mode
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/my_llama3_model_eastern_caverns")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/my_llama3_model_eastern_caverns",
    load_in_4bit=True,
    max_seq_length=2048,
)

# Run a quick test
print("\n------ Welcome to the Eastern caverns Agent! -------\n")
inputs = tokenizer("Explain the function of the QuantumLatch protocol.?", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<ipython-input-2-0f803a1d3a5d>:6: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.5.1 patched 16 layers with 16 QKV layers, 16 O layers and 0 MLP layers.



------ Welcome to the Eastern caverns Agent! -------

Explain the function of the QuantumLatch protocol.? QuantumLatch ensures atomic state transitions across distributed controllers using a 2-phase commit with vector clocks.

## Step 1: Describe the QuantumLatch protocol's goal.
The QuantumLatch protocol aims to ensure atomic state transitions across distributed controllers using a 2-phase commit with vector clocks.

## Step 2: Explain the 2-phase commit mechanism.
The 2-phase commit mechanism ensures atomic state transitions by committing changes in both phases. Vector clocks enable distributed controllers to coordinate their clocks.

## Step 3:
